In [1]:
from IJCAI2017_TOOL import *

In [2]:
Day_list =  [(lambda x:('D'+ str(x).zfill(2))) (x)  for x in range(1,15)]
result_columns =['SHOP_ID'] + Day_list

In [6]:
result_merge1 = pd.read_csv('./Flamingo_Rank4_code/model/xgb_model1.csv',names = result_columns)
result_merge2 = pd.read_csv('./Flamingo_Rank4_code/model/GBDT_model.csv',names = result_columns)
result_merge3 = pd.read_csv('./Flamingo_Rank4_code/model/xgb_model2.csv',names = result_columns)

In [7]:
#三个提升模型的融合 
result_merged  = result_merge1.copy()
result_merged[Day_list] = 0.47* result_merge1[Day_list] + 0.19*result_merge2[Day_list]  + 0.34*result_merge3[Day_list]
result_merged['SHOP_ID'] = result_merged['SHOP_ID'].astype(int)

In [8]:
result = result_merged.copy()

In [9]:
print( len( result_merge1.groupby(['SHOP_ID'],as_index = False).count()) )
print( len( result_merge2.groupby(['SHOP_ID'],as_index = False).count()) )
print( len( result_merge3.groupby(['SHOP_ID'],as_index = False).count()) )
print( len( result_merged.groupby(['SHOP_ID'],as_index = False).count()) )

2000
2000
2000
2000


In [14]:
#加入均值模型
Cor_model = pd.read_csv('./Flamingo_Rank4_code/model/cor_model.csv')
Day_list2 =  [(lambda x:('Cor_D'+ str(x).zfill(2))) (x)  for x in range(1,15)]
Cor_model.columns = ['SHOP_ID'] + Day_list2 + ['credit']

Cor_model['SHOP_ID'] = Cor_model['SHOP_ID'].astype(int)

result = pd.merge(result,Cor_model,on = ['SHOP_ID'],how = 'left' )

#将有融合系数的行进行融合,没有的还是按提升模型的值
null_ind = result['credit'].isnull()
tt1 = result.loc[~null_ind, Day_list].multiply(1.0-result.loc[~null_ind, 'credit'],axis = 0)
tt2 = result.loc[~null_ind, Day_list2].multiply(result.loc[~null_ind, 'credit'],axis = 0)
tt2.columns = Day_list
tt3 = tt1 + tt2

for ind,value in tt3.iterrows():
    result.loc[ind,Day_list] = value.values
result = result[result_columns]

In [15]:
#加入双11修正模型
DOU11 = pd.read_csv('./Flamingo_Rank4_code/model/DOU11_coef.csv')

#result = pd.merge(result, SHOP_INFO_EN, on=['SHOP_ID'],how = 'left')
result = pd.merge(result, DOU11, on=['SHOP_ID'],how = 'left')

#双11系数进行销量调制 双11当天销量乘以双11销量修正模型得到的销量增量，11-12, 11-13由于为周六周日，有理由相信其销量与11-11(周五)的表现存在相似性， 因而乘以0.2及0.1倍的销量增量系数。
result['D11'] = result['D11'] * (result['DOU11']*0.8 + 1.0*(0.2) )
result['D12'] = result['D12'] * (result['DOU11']*0.2+ 1.0*0.8)
result['D13'] = result['D13'] * (result['DOU11']*0.1+ 1.0*0.9)

result = result[result_columns]

In [16]:
result = result.round()
result = result.astype(np.int)
filename = 'FINAL_RESULT.csv'
result.to_csv(filename,header=None, index = False)